In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

input_folder = os.path.join(os.getcwd(), "...")
output_folder = os.path.join(os.getcwd(), "..._dft_images")

os.makedirs(output_folder, exist_ok=True)

FRAME_SIZE = (256, 256)

def process_video(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    sum_dft = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, FRAME_SIZE, interpolation=cv2.INTER_AREA)

        dft = cv2.dft(np.float32(resized_frame), flags=cv2.DFT_COMPLEX_OUTPUT)
        dft_shift = np.fft.fftshift(dft)
        magnitude_spectrum = np.log1p(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))

        if sum_dft is None:
            sum_dft = magnitude_spectrum
        else:
            sum_dft += magnitude_spectrum

        frame_count += 1

    cap.release()

    if frame_count > 0:
        avg_dft = sum_dft / frame_count
        dft_normalized = cv2.normalize(avg_dft, None, 0, 255, cv2.NORM_MINMAX)
        dft_image = np.uint8(dft_normalized)
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        output_path = os.path.join(output_folder, f"{video_name}_dft.png")
        cv2.imwrite(output_path, dft_image)

video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]

for video_file in tqdm(video_files, desc="Processing Videos"):
    video_path = os.path.join(input_folder, video_file)
    process_video(video_path, output_folder)

Processing Videos: 100%|██████████| 1000/1000 [3:11:18<00:00, 11.48s/it] 
